In [1]:
# %load_ext autoreload
# %autoreload 2

In [2]:
import os

import pandas as pd

from bankreg import BankReg

from sources.banktrack.banktrack import Banktrack
from sources.bocc.bocc import Ran
from sources.gabv.gabv import Gabv
from sources.fairfinance.fairfinance import Fairfinance
from sources.switchit.switchit import Switchit
from sources.marketforces.marketforces import Marketforces
from sources.custombank.custombank import Custombank
from sources.wikidata.wikidata import Wikidata

from airtableutils import BankGreenAirtable

BankReg.__instance__ = None
bankreg = BankReg()

In [3]:
%%time

# Banktrack
Banktrack.load_and_create(bankreg)
print('Banktrack Added. New Length: ' + str(len(bankreg.reg)))

# BOCC
Ran.load_and_create(bankreg)
print('BOCC Added. New Length: ' + str(len(bankreg.reg)))

# GABV & B-Impact
Gabv.load_and_create(bankreg)
print('GABV and B-Impact Added. New Length: ' + str(len(bankreg.reg)))

# Fair Finance
Fairfinance.load_and_create(bankreg)
print('Fairfinance Added. New Length: ' + str(len(bankreg.reg)))

# Switchit
Switchit.load_and_create(bankreg)
print('Switchit Added. New Length: ' + str(len(bankreg.reg)))

# Marketforces
Marketforces.load_and_create(bankreg)
print('Marketforces Added. New Length: ' + str(len(bankreg.reg)))

# Wikidata
Wikidata
Wikidata.load_and_create(bankreg)
print('Wikidata Added. New Length: ' + str(len(bankreg.reg)))

# Custom bank
Custombank.load_and_create(bankreg)
print('Custom Data Added. New Length: ' + str(len(bankreg.reg)))

Banktrack Added. New Length: 218
BOCC Added. New Length: 218
GABV and B-Impact Added. New Length: 316
Fairfinance Added. New Length: 368
Switchit Added. New Length: 390
Marketforces Added. New Length: 492
Wikidata Added. New Length: 1766
Custom Data Added. New Length: 1798
CPU times: user 22.3 s, sys: 206 ms, total: 22.5 s
Wall time: 23.8 s


# Manual Cleanup

In [4]:
# generic triodos needs to be removed because it is duplicated in country-specific instances
bankreg.reg.pop('triodos', None)
bankreg.reg.pop('triodos_bank', None)

<triodos_bank <class 'bank.Bank'> with banktrack, fairfinance, wikidata>

In [5]:
df = bankreg.return_registry_as_df()

In [6]:
df

,tag,name,aliases,country,data_sources,website,rating,reason,subsidiary_of,Rank - Total,total-USD,total-EUR,total-GBP,total-AUD,total-CAD
0,bnp_paribas,BNP Paribas,"banque nationale de paris bnp paribas sa,bnp p...","Belgium,France","banktrack,fairfinance,ran,wikidata",http://www.bnpparibas.com/en,worst,5.0% of BNP Paribas's assets in have been inve...,None,4,120.825,106.485787,93.117161,168.269287,159.752270
1,abn_amro,ABN AMRO,"abn amro,abn amro abn amro bank nv abn amro ca...",Netherlands,"banktrack,custom,fairfinance,wikidata",http://www.abnamro.nl/en/index.html,bad,,,,120.825,106.485787,93.117161,168.269287,159.752270
2,anz,ANZ,anz,Australia,"banktrack,marketforces,ran",http://www.anz.com.au,bad,this bank has not divested from fossil fuels,None,42,15.227,13.371884,11.658871,20.892952,20.032603
3,banco_bradesco,Banco Bradesco,"banco bradesco,bradesco",Brazil,"banktrack,fairfinance",https://www.bradescori.com.br/en/,unk,We do not have enough information to rate this...,None,,15.227,13.371884,11.658871,20.892952,20.032603
4,banco_do_brasil,Banco do Brasil,"banco do brasil,bank of brazil banco do brasil",Brazil,"banktrack,fairfinance",http://www.bb.com.br,unk,We do not have enough information to rate this...,None,,15.227,13.371884,11.658871,20.892952,20.032603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1791,tsb,TSB,tsb,New Zealand,custom,None,great,,,,29.459,25.950429,22.637891,40.469105,38.770754
1792,cooperative_nz,Co-operative Bank NZ,cooperative bank nz,New Zealand,custom,None,great,,,,29.459,25.950429,22.637891,40.469105,38.770754
1793,n26,N26,n26,"Austria,Belgium,Denmark,Estonia,Finland,France...",custom,n26.com,great,N26 Invests customer money in unspecified bank...,,,29.459,25.950429,22.637891,40.469105,38.770754
1794,allied_irish_bank,Allied Irish Bank,allied irish bank,Ireland,custom,None,bad,,,,29.459,25.950429,22.637891,40.469105,38.770754


In [8]:
df=bankreg.return_registry_as_df()

df[df['data_sources'] != 'wikidata'].to_csv('not_wikidata.csv')
df[df['data_sources'] == 'wikidata'].to_csv('only_wikidata.csv')

In [9]:
bankreg.reg['wells_fargo'].countries

['Ireland', 'United States']

In [13]:
# this is code that produces supplemental data for the nametag map.
# Roughly, it matches websites against eachother and assigns existing tags
# when the websites match a new data source

df=bankreg.return_registry_as_df()
unique_websites = duped.website.unique()
unique_websites = [w for w in unique_websites if w is not None]
duped = df[df.website.duplicated(keep=False)]
nametag_map = {}

for website in unique_websites:
    rows = duped[duped.website == website]

    wikidata_rows = rows[rows['data_sources'] == 'wikidata']
    non_wikidata_rows = rows[rows['data_sources'] != 'wikidata']
    wikidata_names = wikidata_rows.name
    
    # case if a bank is in the nametag that is not from wikidata:
    if len(non_wikidata_rows) > 0:
        non_wikidata_tag = non_wikidata_rows.tag.iloc[0]
        
        # enter all tags into a nametag dict
        for name in wikidata_names:
            nametag_map[name] = non_wikidata_tag
        
    # case if the bank only exists in wikidata:
    else:
        first_tag = wikidata_rows.tag.iloc[0]
        
        # enter all banks into the dict with the first arbitrary tag
        for name in wikidata_names:
            nametag_map[name] = first_tag
        
nametag_map
   

# Airtable

In [13]:
bg_at = BankGreenAirtable(table_name="staging", local_df=bankreg.return_registry_as_df(), preservation_columns=['name', 'website', 'subsidiary_of'])

In [14]:
bg_at.airtable_backup()

'./airtable_backups/2021.26.04 17.26.52 staging.pkl'

In [15]:
flushed = bg_at.airtable_flush()
print('Num Flushed: ' + str(len(flushed)))

inserted = bg_at.airtable_insert()
print('Num Inserted: ' + str(len(inserted)))

updated = bg_at.airtable_update()
print('Num Updated: ' + str(len(updated)))

Num Flushed: 0
Num Inserted: 2
Num Updated: 1794


In [16]:
inserted

[{'id': 'recOalcS3VkIIPGqT',
  'fields': {'tag': 'line_bank',
   'name': 'LINE Bank',
   'country': ['Taiwan'],
   'rating': 'unk',
   'reason': 'We do not have enough information to rate this bank',
   'data_sources': ['wikidata'],
   'aliases': 'line bank',
   'total-USD': 29.459,
   'total-EUR': 25.950429491835706,
   'total-GBP': 22.637891370261894,
   'total-AUD': 40.46910469475687,
   'total-CAD': 38.7707539492186},
  'createdTime': '2021-04-26T15:26:53.000Z'},
 {'id': 'recIivcBim9e6AKit',
  'fields': {'tag': 'bank_of_spain_building_in_pontevedra',
   'name': 'Bank of Spain building in Pontevedra',
   'country': ['Spain'],
   'rating': 'unk',
   'reason': 'We do not have enough information to rate this bank',
   'data_sources': ['wikidata'],
   'aliases': 'bank of spain building in pontevedra',
   'total-USD': 29.459,
   'total-EUR': 25.950429491835706,
   'total-GBP': 22.637891370261894,
   'total-AUD': 40.46910469475687,
   'total-CAD': 38.7707539492186},
  'createdTime': '2021